# VLM Erax

In [1]:
!python -m pip install -q git+https://github.com/huggingface/transformers accelerate
!python -m pip install -q qwen-vl-utils
!pip install flash-attn -q --no-build-isolation

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 49.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 52.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done


In [2]:
!pip install -qU gdown
!gdown --id 1ZNWz-exIqUm-EJqNQM2_-6Wb_Br0XGhj
!unzip /kaggle/working/samples.zip -d /kaggle/working

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1ZNWz-exIqUm-EJqNQM2_-6Wb_Br0XGhj
From (redirected): https://drive.google.com/uc?id=1ZNWz-exIqUm-EJqNQM2_-6Wb_Br0XGhj&confirm=t&uuid=09786d21-94ae-4b12-a8a5-64be8ac7e010
To: /kaggle/working/samples.zip
100%|██████████████████████████████████████| 27.9M/27.9M [00:00<00:00, 28.3MB/s]
Archive:  /kaggle/working/samples.zip
   creating: /kaggle/working/samples/
   creating: /kaggle/working/samples/idcard/
  inflating: /kaggle/working/samples/idcard/0fb04a58-c484-42ab-96bd-f8d8750e4c55_091074011629_IDFront_jpg.rf.2732aa3ab85d162eeb596fc910025edb.jpg  
  inflating: /kaggle/working/samples/idcard/0fb04a58-c484-42ab-96bd-f8d8750e4c55_091074011629_IDFront_jpg.rf.dc5365dacf34bff5605c25b218f4f02a.jpg  
  inflating: /ka

In [3]:
import os
import base64
import json

import cv2
import numpy as np
import matplotlib.pyplot as  plt

import torch
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

model_path = "erax-ai/EraX-VL-7B-V2.0-Preview"

model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    attn_implementation="eager", # replace with "flash_attention_2" if your GPU is Ampere architecture
    device_map="auto",
    # offload_state_dict=True
)

tokenizer =  AutoTokenizer.from_pretrained(model_path)
# processor = AutoProcessor.from_pretrained(model_path)

min_pixels = 256 * 28 * 28
max_pixels = 1280 * 28 * 28
processor = AutoProcessor.from_pretrained(
     model_path,
     min_pixels=min_pixels,
     max_pixels=max_pixels,
 )

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [10]:
image_path = "/kaggle/working/samples/idcard/0_jpg.rf.6c3ba6fef26ebc2fb8e288cb30a9db9e.jpg"

with open(image_path, "rb") as f:
    encoded_image = base64.b64encode(f.read())
decoded_image_text = encoded_image.decode('utf-8')
base64_data = f"data:image;base64,{decoded_image_text}"

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": base64_data,
            },
            {
                "type": "text",
                "text": "Trích xuất thông tin nội dung từ hình ảnh được cung cấp."
            },
        ],
    }
]

# Prepare prompt
tokenized_text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

image_inputs, video_inputs = process_vision_info(messages)

inputs = processor(
    text=[ tokenized_text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Generation configs
generation_config =  model.generation_config
generation_config.do_sample   = True
generation_config.temperature = 0.01
generation_config.top_k       = 1
generation_config.top_p       = 0.001
#generation_config.min_p       = 0.1
generation_config.best_of     = 1
generation_config.max_new_tokens     = 2048
generation_config.repetition_penalty = 1.01

In [11]:
%%time
generated_ids = model.generate(**inputs, generation_config=generation_config)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

print(output_text[0])

Hình ảnh là một thẻ căn cước công dân của Việt Nam. Ở góc trên bên trái là quốc huy Việt Nam, bên phải là mã QR. Phía trên cùng là dòng chữ "CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM", bên dưới là dòng chữ "Độc lập - Tự do - Hạnh phúc" và dòng chữ tiếng An "SOCIAL REPUBLIC OF VIET NAM" và "Independence - Freedom - Happiness". Tiếp theo là dòng chữ "CĂN CƯỚC CÔNG DÂN" và "Citizen Identity Card". Ở giữa là thông tin cá nhân gồm: Số / No.: 320495186074, Họ và tên / Full name: VU THỊ BÍCH, Ngày sinh / Date of birth: 30/04/1991, Giới tính / Sex: Nữ, Quốc tịch / Nationality: Việt Nam, Quê quán / Place of origin: Vĩnh Bảo, Hải Phòng, Nơi thường trú / Place of residence: 17/9/2, Trần Nguyễn Hân, Vĩnh Bảo, Hải Phòng. Ở góc dưới bên trái là dòng chữ "Có giá trị đến: 30/04/2041" và "Date of expiry".
CPU times: user 26.2 s, sys: 0 ns, total: 26.2 s
Wall time: 26.2 s


# QA Agents

In [2]:
!pip install -q ollama gradio
!curl -fsSL https://ollama.com/install.sh | sh

    return future.result()
  File "<ipython-input-1-3938b4666513>", line 30, in start_ollama_serve
  File "<ipython-input-1-3938b4666513>", line 9, in run_process
  File "/usr/lib/python3.10/asyncio/subprocess.py", line 218, in create_subprocess_exec
    transport, protocol = await loop.subprocess_exec(
  File "/usr/lib/python3.10/asyncio/base_events.py", line 1681, in subprocess_exec
    transport = await self._make_subprocess_transport(
  File "/usr/lib/python3.10/asyncio/unix_events.py", line 207, in _make_subprocess_transport
    transp = _UnixSubprocessTransport(self, protocol, args, shell,
  File "/usr/lib/python3.10/asyncio/base_subprocess.py", line 36, in __init__
    self._start(args=args, shell=shell, stdin=stdin, stdout=stdout,
  File "/usr/lib/python3.10/asyncio/unix_events.py", line 799, in _start
    self._proc = subprocess.Popen(
  File "/usr/lib/python3.10/subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "/usr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 28.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 100.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.5 MB/s eta 0:00:00
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
import asyncio
import threading
import socket


def run_ollama_server():
    async def run_process(cmd):
        print('>>> starting', *cmd)
        process = await asyncio.create_subprocess_exec(
            *cmd,
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE,
            # env={**os.environ, 'OLLAMA_NUM_PARALLEL': '8', 'OLLAMA_MAX_LOADED_MODELS': '1'}
        )

        # define an async pipe function
        async def pipe(lines):
            async for line in lines:
                print(line.decode().strip())

            await asyncio.gather(
                pipe(process.stdout),
                pipe(process.stderr),
            )

        # call it
        await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

    async def start_ollama_serve():
        await run_process(['ollama', 'serve'])

    def run_async_in_thread(loop, coro):
        asyncio.set_event_loop(loop)
        loop.run_until_complete(coro)
        loop.close()

    # Create a new event loop that will run in a new thread
    new_loop = asyncio.new_event_loop()

    # Start ollama serve in a separate thread so the cell won't block execution
    thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
    thread.start()
run_ollama_server()

>>> starting ollama serve


In [4]:
import ollama
# ollama.pull('phi4:14b-q8_0') 16GB RAM + 8GBx2 GPU
# ollama.pull("llama3.1:8b-instruct-fp16")
ollama.pull("llama3.2:3b-instruct-q4_0")
# ollama.pull("deepseek-r1:32b-qwen-distill-q8_0")

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is:

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIF4U0h/CEXSo8aPin/kMGYZAi/ANNCunX0FUKQStL2En

2025/02/19 11:04:50 routes.go:1186: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* f

ProgressResponse(status='success', completed=None, total=None, digest=None)

### Chatbot

In [ ]:
LLM_NAME="llama3.2:3b-instruct-q4_0"

In [ ]:
import gradio as gr
import ollama


def chat_with_model(message, history):
    # Format lịch sử chat theo chuẩn Ollama
    formatted_history = []
    for human, ai in history[-10:]:
        formatted_history.append({"role": "user", "content": human})
        formatted_history.append({"role": "assistant", "content": ai})
    formatted_history.append({"role": "user", "content": message})


    # Tạo stream response từ model
    response = ollama.chat(
        # model="llama3.1:8b-instruct-fp16",
        model=LLM_NAME,
        messages=formatted_history,
        stream=True
    )

    full_response = ""
    for chunk in response:
        part = chunk["message"]["content"]
        full_response += part
        yield full_response

# Tạo giao diện Gradio
demo = gr.ChatInterface(
    fn=chat_with_model,
    title="DeepSeek-R1 Chatbot",
    examples=["Giải thích về AI", "Viết một bài thơ ngắn", "Cách làm bánh flan?"],
    theme="soft"
).launch(share=True, show_error=True)

### Voicebot

In [5]:
from ollama import Client, AsyncClient

# --------------------- CONSTANT ---------------------
VOICE_API_URL = "http://127.0.0.1:5000/api/voice"
OLLAMA_CLIENT = AsyncClient(
    host='http://localhost:11434',
    headers={'x-some-header': 'some-value'}
)
LLM_NAME = "llama3.2:3b-instruct-q4_0"

In [ ]:
import io
import os
import tempfile
import numpy as np
import soundfile as sf
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.responses import JSONResponse
from transformers import pipeline
from typing import Optional
import uvicorn
import requests
import gradio as gr
from typing import List, Generator, AsyncGenerator

# --------------------- Phần 1: Xử lý âm thanh ---------------------
# Tạo pipeline chuyển đổi giọng nói thành văn bản (ASR) sử dụng mô hình "vinai/PhoWhisper-small"
# Mô hình này được xây dựng dựa trên ý tưởng của Whisper (OpenAI, "Whisper: Robust Speech Recognition via Large-Scale Weak Supervision")
# Dữ liệu âm thanh được biểu diễn dưới dạng np.array, ví dụ: np.array([...], dtype=float32) với shape (n_samples,) hoặc (n_samples, n_channels)
# Khởi tạo mô hình Whisper sử dụng CPU (device=-1) để tránh lỗi CUDA
transcriber = pipeline(
    "automatic-speech-recognition",
    model="vinai/PhoWhisper-large",
    device="cuda"
)

app = FastAPI()

def speech_to_text_whisper(audio: np.ndarray, sample_rate: int = 16000) -> str:
    result = transcriber({"sampling_rate": sample_rate, "raw": audio}, chunk_length_s=30)
    return result.get("text", "")

@app.post("/api/voice")
async def voice_api(file: Optional[UploadFile] = File(None)):
    if file is None or file.filename == "":
        raise HTTPException(status_code=400, detail="Không nhận được file âm thanh")

    temp_file_path = ""
    try:
        # Tạo file tạm với hậu tố (suffix) phù hợp với định dạng của file upload
        suffix = os.path.splitext(file.filename)[1]
        with tempfile.NamedTemporaryFile(delete=False, suffix=suffix) as tmp:
            temp_file_path = tmp.name
            contents = await file.read()
            tmp.write(contents)
        # Lấy đường dẫn tuyệt đối của file tạm
        abs_path = os.path.abspath(temp_file_path)
        # Đọc file âm thanh từ đường dẫn tuyệt đối
        audio_data, sr = sf.read(abs_path, dtype='float32')
        if audio_data.dtype == np.int16:
            audio_data = audio_data.astype(np.float32) / 32768.0
        text = speech_to_text_whisper(audio_data, sample_rate=sr)
        return JSONResponse(content={"text": text, "file_path": abs_path}, status_code=200)
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Lỗi xử lý: {str(e)}")
    finally:
        if temp_file_path and os.path.exists(temp_file_path):
            os.remove(temp_file_path)

# --------------------- Phần 1: Chatbot tích hợp ---------------------
async def chat_with_model(message: str, history: List[List[str]]) -> AsyncGenerator[List[List[str]], None]:
    """
    Gọi mô hình chat (Ollama) với lịch sử hội thoại hiện tại và tin nhắn mới.
    Input:
      - message: tin nhắn của người dùng (str)
      - history: danh sách lịch sử chat, mỗi phần tử [tin nhắn người dùng, phản hồi AI]
    Output:
      - Generator trả về lịch sử chat được cập nhật theo từng phần của phản hồi.
    """
    # Chuẩn hóa lịch sử chat theo định dạng của Ollama
    formatted_history = []
    for human, ai in history[-10:]:
        formatted_history.append({"role": "user", "content": human})
        formatted_history.append({"role": "assistant", "content": ai})
    formatted_history.append({"role": "user", "content": message})

    # # Gọi hàm chat của Ollama với stream=True để nhận phản hồi từng phần
    # response = ollama.chat(
    #     model="llama3.2:3b-instruct-q4_0",
    #     messages=formatted_history,
    #     stream=True
    # )
    response = await OLLAMA_CLIENT.chat(
        model=LLM_NAME,
        messages=formatted_history,
        stream=True
    )

    full_response = ""
    async for chunk in response:
        part = chunk["message"]["content"]
        full_response += part
        # Cập nhật lịch sử chat theo thời gian thực
        updated_history = history + [[message, full_response]]
        yield updated_history


def send_audio_to_api(audio_file_path: str) -> str:
    """
    Gửi file âm thanh đến API FastAPI để chuyển đổi thành văn bản.
    Input:
      - audio_file_path: đường dẫn file âm thanh (str)
    Output:
      - Văn bản chuyển đổi (str) hoặc thông báo lỗi.
    """
    if not audio_file_path:
        return ""
    try:
        with open(audio_file_path, "rb") as file:
            files = {"file": (audio_file_path.split("/")
                              [-1], file, "audio/wav")}
            response = requests.post(VOICE_API_URL, files=files)
            response.raise_for_status()
            data = response.json()
            return data.get("text", "")
    except requests.exceptions.RequestException as e:
        print(f"Lỗi gửi audio đến API: {e}")
        return f"Lỗi: {e}"
    except Exception as e:
        print(f"Lỗi không xác định: {e}")
        return f"Lỗi: {e}"


# --------------------- Phần 2: Giao diện Gradio ---------------------
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Voicebot demo")
    examples = gr.Examples(
        examples=[
            "Giải thích về AI",
            "Viết một bài thơ ngắn",
            "Cách làm bánh flan?"
        ],
        inputs=[gr.Textbox(label="Ví dụ", visible=False)]
    )

    with gr.Row():
        # Cho phép nhập âm thanh từ microphone, trả về đường dẫn file tạm thời
        audio_input = gr.Audio(
            sources=["microphone"], type="filepath", label="Input audio (optional)")
        text_output_from_audio = gr.Textbox(label="Text recognized", interactive=False)

    # Khi có sự thay đổi từ input âm thanh, gửi file đến API để chuyển đổi
    audio_input.change(fn=send_audio_to_api, inputs=audio_input,
                       outputs=text_output_from_audio)

    async def process_input(message: str, history: List[List[str]], transcribed_text: str) -> AsyncGenerator[List[List[str]], None]:
        """
        Xử lý đầu vào: ưu tiên dùng văn bản chuyển từ âm thanh nếu có, ngược lại dùng tin nhắn nhập tay.
        Gọi hàm chat_with_model để nhận phản hồi từng phần từ mô hình.
        """
        user_input = transcribed_text if transcribed_text else message
        response_generator = chat_with_model(user_input, history)
        async for updated_history in response_generator:
            yield updated_history, None, None, None

    text_input = gr.Textbox(label="Input text...")
    submit_btn = gr.Button("Submit")
    submit_btn.click(fn=process_input, inputs=[
                     text_input, chatbot, text_output_from_audio],
                     outputs=[chatbot, text_input, text_output_from_audio, audio_input])

# --------------------- Phần 3: Chạy FastAPI và Gradio song song ---------------------
def run_fastapi():
    uvicorn.run(app, host="127.0.0.1", port=5000)


# --------------------- Chạy FastAPI trên một luồng riêng để không làm gián đoạn Gradio ---------------------
fastapi_thread = threading.Thread(target=run_fastapi, daemon=True)
fastapi_thread.start()
demo.launch(share=True, server_port=8000, show_error=True)

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.17G [00:00<?, ?B/s]